Author: Rudi Kreidenhuber, <Rudi.Kreidenhuber@gmail.com>, 
License: BSD (3-clause)

## To do:

Create Tables for single seizure (just like grand average) ?

EKG?

Create a radar chart of EEG and Semiology signs (r/l hemisphere: front, temp, parietal, occipital) - https://plotly.com/python/radar-chart/



----

# Video EEG Monitoring Annotation visualizer

----


## Inputs:
 - .edf-files you wish to analyze go into ./data folder

## Run:
 - Press play :-)

## Outputs:
 - Found in results folder
 - Results for single files are put into a folder that matches the input-filename

----

## Howto:
 1. **Mark Events in EEG file using the following prefixes:**
 - e- --> EEG marker
 - s- --> Semiology marker
 - no prefix --> Everything else (clinical tests during/ after seizure)
 - i- --> Marker to ignore for focused analysis

 - One marker **must (!) contain "Beginn"** --> this is considered the seizure onset (if it is missing, onset is set to zero)
 - every marker **can** contain Beginn, for example:
 - Onset first seen in EEG --> Markername "e-asdBeginnfgh" --> would still be recognized as EEG marker and seizure onset
 2. **Save EEG file in .edf format and copy to ./data folder**
 - Every file in this folder is going to be analyzed, if it ends with .edf
 

----
## Configuration
----

### Parameters:
graph_sep_line_width
- How far blue dashed seperator lines are apart from each other

plot_tmin
- First time point in seconds from onset, that should be included in the visualization, set to 0 to deactivate

plot_tmax
- Last time point in seconds from onset, that should be included in the visualization, set to 0 to deactivate


In [1]:
subj_name = "Test-Patient"
graph_sep_line_width = 5
plot_tmin = -20
plot_tmax = 100

----
## Pipeline start
----

In [2]:
# import everything
import os
from os.path import join
import glob
import mne
from mne import Report
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import plotly.graph_objects as go
from utils import (get_parent_dir, extract_lab_sec, loc_of_sep_lines, plot_seizure_horizontal, 
                                        plot_seizure_vertical, raw_to_df, extract_groups, 
                                        extract_ordered_groups, save_plotly_to_html,
                                        shrink_df_to_tmax, create_results_folders, 
                                        save_fig_to_disc, plot_interactive_subplot_with_table,
                                        extract_parameters_from_raw, plot_eventcounts, plot_interactive_tables,
                                        plot_interactive_eeg_and_semio, plot_interactive_eventcount,
                                        plot_interactive_testing_results, win_save_fig_to_disc, win_create_results_folders)
import plotly as py
import ipywidgets as widgets
import plotly.io as pio

py.offline.init_notebook_mode(connected=True)

win = True

# no need to show figures here
plt.ioff() 

# grab .edfs
edfs = glob.glob("../data/*.edf")
if win:
    edfs = glob.glob("..\\data\\*.edf")

print("edfs found:\n", edfs)

report = Report(subject=subj_name, title="Event summary")

edfs found:
 ['..\\data\\PC19012021_F1.edf', '..\\data\\PC19012021_F2.edf', '..\\data\\PC19012021_F3.edf']
Embedding : jquery.js
Embedding : jquery-ui.min.js
Embedding : bootstrap.min.js
Embedding : jquery-ui.min.css
Embedding : bootstrap.min.css


----
## Visualization
----

In [3]:
if len(edfs) > 0:
    # Create folder structure
    if win:
        win_create_results_folders(edfs)
    else:
        create_results_folders(edfs)
    df = dict() 
    shrinked_df = dict()
    for e in edfs:
        print(f"Now processing file: {e}")
        raw = mne.io.read_raw(e, preload=True)                      #read
        df[e], onset = raw_to_df(raw, e)                            # annotations to DataFrame
        if win:
            e_events, s_events, t_events = extract_ordered_groups(df[e], e.split("\\")[-1])
        else:
            e_events, s_events, t_events = extract_ordered_groups(df[e], e.split("/")[-1])     # Extract groups

        # save
        
        #### windows paths...
        if win:
            source = e.split("\\")[-1].split(".")[0]
            save_path = join("..", "results", source, "tables", "All_events.tsv")
            df[e].to_csv(save_path, sep="\t")
            save_path = join("..", "results", source, "tables", "EEG_events.tsv")
            e_events.to_csv(save_path, sep="\t")
            save_path = join("..", "results", source, "tables", "Semiology_events.tsv")
            s_events.to_csv(save_path, sep="\t")
            save_path = join("..", "results", source, "tables", "Testing_events.tsv")
            t_events.to_csv(save_path, sep="\t")

        
        else:
            source = e.split("/")[-1].split(".")[0]
            save_path = ("../results/" + source + "/" + "tables/All_events.tsv")
            df[e].to_csv(save_path, sep="\t")
            save_path = ("../results/" + source + "/" + "tables/EEG_events.tsv")
            e_events.to_csv(save_path, sep="\t")
            save_path = ("../results/" + source + "/" + "tables/Semiology_events.tsv")
            s_events.to_csv(save_path, sep="\t")
            save_path = ("../results/" + source + "/" + "tables/Test_events.tsv")
            t_events.to_csv(save_path, sep="\t")
                
        # Shrink df to tmax
        if df[e].iloc[0,0]:
            print(f"\nCalculating parameters for focused visualization, tmin = {plot_tmin}, tmax = {plot_tmax} s.\n\n\n")
            
            shrinked_df[e] = shrink_df_to_tmax(df=df[e], tmax=plot_tmax, tmin=plot_tmin)
            se_events, ss_events, st_events = extract_groups(shrinked_df[e], e)

            # Visualizations
            # vertical static
            seizure_vertical_w_limits = plot_seizure_vertical(df=shrinked_df[e], eeg=se_events, 
                                                                semio=ss_events, testing=st_events, 
                                                                source=e.split("/")[-1], tmin= plot_tmin, 
                                                                tmax=plot_tmax, name="seizure vertical with time limits",
                                                                graph_sep_line_width=graph_sep_line_width)
            if win:
                win_save_fig_to_disc(seizure_vertical_w_limits, e, "seizure_vertical_with_limits")
            else:
                save_fig_to_disc(seizure_vertical_w_limits, e, "seizure_vertical_with_limits")

            # horizontal static
            seizure_horizontal_w_limits = plot_seizure_horizontal(df=shrinked_df[e], eeg=se_events, 
                                                                semio=ss_events, testing=st_events, 
                                                                source=e.split("/")[-1], tmin= plot_tmin, 
                                                                tmax=plot_tmax, name="seizure horizontal with time limits",
                                                                graph_sep_line_width=graph_sep_line_width)
            if win:
                win_save_fig_to_disc(seizure_vertical_w_limits, e, "seizure_horizontal_w_limits")
            else:           
                save_fig_to_disc(seizure_horizontal_w_limits, e, "seizure_horizontal_w_limits")

            # event counts (plot.ly)
            event_counts = plot_interactive_eeg_and_semio(eeg=se_events, semio=ss_events, 
                                                                source=e.split("/")[-1])
            cap = source + " VIZ --> event counts (with limits)"
            sec = source
            report.add_htmls_to_section(event_counts.to_html(full_html=False), section=sec, captions=cap)


        # vertical static
        seizure_vertical = plot_seizure_vertical(df=df[e], eeg=e_events, semio=s_events, 
                                                    testing=t_events, source=e.split("/")[-1], 
                                                    name="seizure vertical",
                                                    graph_sep_line_width=graph_sep_line_width)
        if win:
            win_save_fig_to_disc(seizure_vertical_w_limits, e, "seizure_vertical")
        else:           
            save_fig_to_disc(seizure_vertical, e, "seizure_vertical")

        # horizontal static
        seizure_horizontal = plot_seizure_horizontal(df=df[e], eeg=e_events, semio=s_events, 
                                                    testing=t_events, source=e.split("/")[-1], 
                                                    name="seizure_horizontal",
                                                    graph_sep_line_width=graph_sep_line_width)
        if win:
            win_save_fig_to_disc(seizure_vertical_w_limits, e, "seizure_horizontal")
        else:
            save_fig_to_disc(seizure_horizontal, e, "seizure_horizontal")

        # event counts (plot.ly)
        source=e.split("/")[-1].split(".edf")[0]
        if win:
            source=e.split("\\")[-1].split(".edf")[0]
        event_counts = plot_interactive_eeg_and_semio(eeg=e_events, semio=s_events, source=source)
        cap = source + " VIZ --> event_conuts"
        sec = source
        report.add_htmls_to_section(event_counts.to_html(full_html=False), section=sec, captions=cap)

        # Testing
        cap = source + " VIZ --> Testing results"
        testing_viz = plot_interactive_testing_results(t_events=t_events, title=cap)
        report.add_htmls_to_section(testing_viz.to_html(full_html=False), section=sec, captions=cap)

Now processing file: ..\data\PC19012021_F1.edf
Extracting EDF parameters from c:\Users\User\Desktop\VEEG_Event_Processor-main\data\PC19012021_F1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 122111  =      0.000 ...   476.996 secs...
Error: No marker containing "Beginn" found, cannot determine seizure onset for file:  ..\data\PC19012021_F1.edf
Setting seizure onset to the beginning of the file

Calculating parameters for focused visualization, tmin = -20, tmax = 100 s.



Now processing file: ..\data\PC19012021_F2.edf
Extracting EDF parameters from c:\Users\User\Desktop\VEEG_Event_Processor-main\data\PC19012021_F2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 85247  =      0.000 ...   332.996 secs...
Error: No marker containing "Beginn" found, cannot determine seizure onset for file:  ..\data\PC19012021_F2.edf
Setting seizure onset to the beginning of the file

Calculatin

----
## Save data
----

In [4]:
df_keys = df.keys()
# horizontal grand average
for idx, val in enumerate(df_keys):
    if idx == 0:
        concat = df[val]
        concat["source"] = val
        cols = list(concat)
        if not "source" in concat:
            cols.insert(0, cols.pop(cols.index('source')))
        concat = concat.loc[:, cols]
        print(concat.time_from_onset)
        concat = concat.sort_values(by=["time_from_onset"])
        concat = concat.drop(columns=["onset"], axis=1)
        concat["order_of_occurence"] = (1 + np.arange(len(concat.loc[:,"time_from_onset"])))
        
    if idx > 0:
        new_df = df[val]
        if not "source" in new_df.keys():
            new_df["source"] = val
        cols = list(new_df)
        cols.insert(0, cols.pop(cols.index('source')))
        new_df = new_df.loc[:, cols]
        new_df.drop(columns=["onset"], axis=1, inplace=True)
        new_df["order_of_occurence"] = (1 + np.arange(len(new_df.loc[:,"time_from_onset"]))).astype(int)
        concat = pd.merge(concat, new_df, how="outer", on="description", suffixes=(" ", "  "))
    #print(f"\n\n\n\nRun {idx} --> concat = {concat}")
    idx += 1

if win:
    concat.to_csv("..\\results\\grand_average\\tables\\All_Data_horizontal.tsv", sep="\t")
else:
    concat.to_csv("../results/grand_average/tables/All_Data_horizontal.tsv", sep="\t")

# EEG/Semio/Test horizontal grand_average
for idx, val in enumerate(df_keys):
    if idx == 0:
        eeg_ga, semio_ga, test_ga = extract_ordered_groups(df[val], source = val)
        # this should not be necessary:
        eeg_ga.sort_values(by=["time_from_onset"])
        semio_ga.sort_values(by=["time_from_onset"])
        test_ga.sort_values(by=["time_from_onset"])
        
    if idx > 0:
        new_df = df[val]
        ne, ns, nt = extract_ordered_groups(new_df, source = val)
        #new_df.insert(loc=0, column='source', value=val.split("/")[-1])
        eeg_ga = pd.merge(eeg_ga, ne, how="outer", on="description", suffixes=(" ", "  ")) 
        semio_ga = pd.merge(semio_ga, ns, how="outer", on="description", suffixes=(" ", "  "))
        test_ga = pd.merge(test_ga, nt, how="outer", on="description", suffixes=(" ", "  "))
    idx += 1

# drop onset columns
to_keep = [c for c in eeg_ga.columns if not c.lower().startswith("onse")]
eeg_ga = eeg_ga[to_keep]
to_keep = [c for c in semio_ga.columns if not c.lower().startswith("onse")]
semio_ga = semio_ga[to_keep]
to_keep = [c for c in test_ga.columns if not c.lower().startswith("onse")]
test_ga = test_ga[to_keep]

if win:
    eeg_ga.to_csv("..\\results\\grand_average\\tables\\EEG_grand_average.tsv", sep="\t")
    semio_ga.to_csv("..\\results\\grand_average\\tables\\Semiology_grand_average.tsv", sep="\t")
    test_ga.to_csv("..\\results\\grand_average\\tables\\Testing_grand_average.tsv", sep="\t")
else:
    eeg_ga.to_csv("../results/grand_average/tables/EEG_grand_average.tsv", sep="\t")
    semio_ga.to_csv("../results/grand_average/tables/Semiology_grand_average.tsv", sep="\t")
    test_ga.to_csv("../results/grand_average/tables/Testing_grand_average.tsv", sep="\t")


# Vertical grand average
for idx, val in enumerate(df_keys):
    if idx == 0:
        concat = df[val]
        concat.loc[:, "source"] = val.split("/")[-1]        
    if idx > 0:
        new_df = df[val]
        new_df.loc[:, "source"] = val.split("/")[-1]
        concat = pd.concat([concat, new_df], axis=0)     
    idx += 1
concat.drop(columns=["onset"], axis=1)
concat.drop(index=4, axis=1, inplace=True)

concat = concat.sort_values(by=["time_from_onset"])

if win:
    concat.to_csv("..\\results\\grand_average\\tables\\All_events.tsv", sep="\t")
else:
    concat.to_csv("../results/grand_average/tables/All_events.tsv", sep="\t")

0       0.00
1     162.36
2     171.36
3     171.40
4     174.24
5     175.97
6     177.37
7     177.43
8     178.57
9     184.70
10    185.34
11    187.43
12    188.38
13    188.71
14    190.98
15    191.68
16    192.21
17    196.65
18    208.65
19    210.45
20    221.80
21    229.30
22    236.20
23    240.69
24    245.32
25    257.46
26    264.38
27    274.11
28    281.21
29    286.62
30    307.15
31    319.16
32    322.43
33    326.28
34    328.96
35    333.35
36    340.30
37    347.13
38    351.52
39    357.25
40    366.04
41    371.74
42    395.43
43    398.16
44    402.96
45    408.64
46    424.96
47    433.49
48    442.88
49    446.58
Name: time_from_onset, dtype: float64


----
# Interactive Visualization
----

In [5]:
# Seizure subplots
event_folders = glob.glob("../results/*")
if win:
    event_folders = glob.glob("..\\results\\*")
data = dict()
interactive_plots = dict()
for e in event_folders:
    if win:
        source = e.split("\\")[-1]
    else:
        source = e.split("/")[-1]
    tsv = join(e, "tables", "All_events.tsv")
    data[source] = pd.read_csv(tsv, sep="\t")
    EEG, Semio, Test = extract_ordered_groups(data[source], source)
    interactive_plots[source] = plot_interactive_subplot_with_table(data[source], EEG, Semio, Test, title=source)
    save_name = join("..", "results", source, "viz", str(source + "_interactive_viz.html"))
    if not os.path.isfile(save_name):
        save_plotly_to_html(interactive_plots[source], source=source)
        cap = source + " --> interactive Viz"
        report.add_htmls_to_section(interactive_plots[source].to_html(full_html=False), section=source, captions=cap)


# grand averages
if win:
    ga_h = pd.read_csv("..\\results\\grand_average\\tables\\All_Data_horizontal.tsv", sep="\t")
    EEG_ga = pd.read_csv("..\\results\\grand_average\\tables\\EEG_grand_average.tsv", sep="\t")
    semio_ga = pd.read_csv("..\\results\\grand_average\\tables\\Semiology_grand_average.tsv", sep="\t")
    test_ga = pd.read_csv("..\\results\\grand_average\\tables\\Testing_grand_average.tsv", sep="\t")
    all_events_vert = pd.read_csv("..\\results\\grand_average\\tables\\All_events.tsv", sep="\t")

else:
    ga_h = pd.read_csv("../results/grand_average/tables/All_Data_horizontal.tsv", sep="\t")
    EEG_ga = pd.read_csv("../results/grand_average/tables/EEG_grand_average.tsv", sep="\t")
    semio_ga = pd.read_csv("../results/grand_average/tables/Semiology_grand_average.tsv", sep="\t")
    test_ga = pd.read_csv("../results/grand_average/tables/Testing_grand_average.tsv", sep="\t")
    all_events_vert = pd.read_csv("../results/grand_average/tables/All_events.tsv", sep="\t")

# Interactive plots
# Grand average - with times
ga_fig = plot_interactive_tables(ga_h, EEG_ga, semio_ga, test_ga)
source="grand_average"
cap = source + " --> Tabular data"
report.add_htmls_to_section(ga_fig.to_html(full_html=False), section=source, captions=cap)

# Grand average - order only
to_keep = [c for c in ga_h.columns if not c.lower().startswith("time")]
to_keep = [c for c in to_keep if not c.lower().startswith("unnam")]
if win:
    ga_h[to_keep].to_csv("..\\results\\grand_average\\tables\\All_events_order_only.csv", sep="\t")
else:
    ga_h[to_keep].to_csv("../results/grand_average/tables/All_events_order_only.csv", sep="\t")
oga_fig = plot_interactive_tables(ga_h[to_keep], EEG_ga[to_keep], semio_ga[to_keep], test_ga[to_keep])
source="grand_average"
cap = source + " --> Tabular data - order only"
report.add_htmls_to_section(oga_fig.to_html(full_html=False), section="grand_average", captions=cap)



# Grand average - Event counts
EEG, Semio, Test = extract_ordered_groups(all_events_vert, source=source)
source = "grand_average"
cap = source + " --> EEG event_counts"
eeg_counts = plot_interactive_eventcount(df=EEG, mode="EEG", source=source)
report.add_htmls_to_section(eeg_counts.to_html(full_html=False), section=source, captions=cap)

cap = source + " --> Semiology event_counts"
semio_counts = plot_interactive_eventcount(df=Semio, mode="Semiology", source=source)
report.add_htmls_to_section(semio_counts.to_html(full_html=False), section=source, captions=cap)

cap = source + " --> Testing event_counts"
test_counts = plot_interactive_eventcount(df=Test, mode="Testing", source=source)
report.add_htmls_to_section(test_counts.to_html(full_html=False), section=source, captions=cap)

cap = source + " --> Testing results"
sec="grand_average"
testing_viz = plot_interactive_testing_results(t_events=Test, title=cap)
report.add_htmls_to_section(testing_viz.to_html(full_html=False), section=sec, captions=cap)

# Save all
report_save_name = "../results/cumulative_report.html"
if win:
    report_save_name = "..\\results\\cumulative_report.html"
report.save(report_save_name, overwrite=True)

Saving report to location C:\Users\User\Desktop\VEEG_Event_Processor-main\results\cumulative_report.html
Rendering : Table of Contents
PC19012021_F1
 ... PC19012021_F1 VIZ --> event counts (with limits)
 ... PC19012021_F1 VIZ --> event_conuts
 ... PC19012021_F1 VIZ --> Testing results
 ... PC19012021_F1 --> interactive Viz
PC19012021_F2
 ... PC19012021_F2 VIZ --> event counts (with limits)
 ... PC19012021_F2 VIZ --> event_conuts
 ... PC19012021_F2 VIZ --> Testing results
 ... PC19012021_F2 --> interactive Viz
PC19012021_F3
 ... PC19012021_F3 VIZ --> event counts (with limits)
 ... PC19012021_F3 VIZ --> event_conuts
 ... PC19012021_F3 VIZ --> Testing results
 ... PC19012021_F3 --> interactive Viz
 ... grand_average --> Testing results
grand_average
 ... grand_average --> interactive Viz
 ... grand_average --> Tabular data
 ... grand_average --> Tabular data - order only
 ... grand_average --> EEG event_counts
 ... grand_average --> Semiology event_counts
 ... grand_average --> Testing e

'C:\\Users\\User\\Desktop\\VEEG_Event_Processor-main\\results\\cumulative_report.html'